In [1]:
from pymongo import MongoClient
from bson.objectid import ObjectId

In [2]:
client = MongoClient("mongodb://172.17.0.2:27017/")

In [3]:
db = client.ecommerce
users = db.users
posts = db.posts
tags = db.tags

In [5]:
users.delete_many({})
posts.delete_many({})
tags.delete_many({})

DeleteResult({'n': 9, 'ok': 1.0}, acknowledged=True)

In [6]:
sample_tags = [
    {"name": "MongoDB"},
    {"name": "Database"},
    {"name": "Beginner"},
    {"name": "PyMongo"},
    {"name": "Programming"},
    {"name": "Optimisation"},
    {"name": "Schema"},
    {"name": "NoSQL"},
    {"name": "SQL"},
]

tags.insert_many(sample_tags)

# Retrieve the tags with their ObjectIds to reference in posts
retrieved_tags = {tag["name"]: tag["_id"] for tag in tags.find()}

In [7]:
sample_users = [
    {"name": "John", "email": "john@doe.com", "blog_posts": []},
    {"name": "Bob", "email": "bob@strode.com", "blog_posts": []},
    {"name": "Alice", "email": "alice@packer.com", "blog_posts": []}
]
users.insert_many(sample_users)

InsertManyResult([ObjectId('6910e44f4108134fdc255368'), ObjectId('6910e44f4108134fdc255369'), ObjectId('6910e44f4108134fdc25536a')], acknowledged=True)

In [8]:
posts.insert_many([
    {
        "title": "How to Use MongoDB",
        "content": "Using MongoDB effectively.",
        "author": "John",
        "comments": [
            {
                "user_name": "Bob",
                "text": "Great post!",
                "timestamp": "2024-09-12T10:00"
            }
        ],
        "tags": [retrieved_tags['MongoDB'], retrieved_tags['Beginner']]
    },
    {
        "title": "Understanding PyMongo",
        "content": "A guide to using PyMongo with MongoDB.",
        "author": "John",
        "comments": [
            {
                "user_name": "Alice",
                "text": "Very informative.",
                "timestamp": "2024-09-13T14:30"
            },
            {
                "user_name": "Bob",
                "text": "Thanks for the tips!",
                "timestamp": "2024-09-13T15:00"
            }
        ],
        "tags": [retrieved_tags['PyMongo'], retrieved_tags['Programming']]
    },
    {
        "title": "Database Optimization",
        "content": "Techniques for optimizing database performance.",
        "author": "Alice",
        "comments": [],
        "tags": [retrieved_tags['Database'], retrieved_tags['Optimisation']]
    },
    {
        "title": "Schema Design in MongoDB",
        "content": "How to model your data in MongoDB.",
        "author": "Bob",
        "comments": [
            {
                "user_name": "Alice",
                "text": "This is helpful!",
                "timestamp": "2024-09-14T09:45"
            }
        ],
        "tags": [retrieved_tags['Schema'], retrieved_tags['MongoDB']]
    },
    {
        "title": "MongoDB vs SQL Databases",
        "content": "Comparing NoSQL and SQL databases.",
        "author": "Alice",
        "comments": [
            {
                "user_name": "Bob",
                "text": "Great comparisons!",
                "timestamp": "2024-09-15T08:20"
            },
            {
                "user_name": "John",
                "text": "Loved this article!",
                "timestamp": "2024-09-15T10:15"
            }
        ],
        "tags": [retrieved_tags['NoSQL'], retrieved_tags['SQL']]
    },
])

InsertManyResult([ObjectId('6910e4504108134fdc25536b'), ObjectId('6910e4504108134fdc25536c'), ObjectId('6910e4504108134fdc25536d'), ObjectId('6910e4504108134fdc25536e'), ObjectId('6910e4504108134fdc25536f')], acknowledged=True)

In [9]:
posts_with_details = posts.find({}, {"title": 1, "author": 1, "_id": 1})

In [10]:
# Updating users blog posts list
for post in posts_with_details:
    users.update_one({"name": post['author']}, {
        "$push": {
            "blog_posts": {
                "blog_id": post["_id"],
                "title": post["title"]
            }
        }
    })

In [12]:
# Create an index on the author field
posts.create_index([("author", 1)])

'author_1'

In [13]:
posts_by_john_with_index = posts.find({"author": "John"})

for post in posts_by_john_with_index:
    print(f"\nPost Title: {post['title']}")


Post Title: How to Use MongoDB

Post Title: Understanding PyMongo


In [14]:
posts.create_index([("comments.user_name", 1)])

'comments.user_name_1'

In [15]:
pipeline = [
    {"$unwind": "$comments"},
    {"$match": {"comments.user_name": "Bob"}},
    {"$project": {"_id": 0, "comment": "$comments", "blogPostTitle": "$title"}}
]

comments_by_user = posts.aggregate(pipeline)

for comment in comments_by_user:
    print(f"Blog Post Title{comment['blogPostTitle']} - Comment {comment['comment']['text']}")

Blog Post TitleHow to Use MongoDB - Comment Great post!
Blog Post TitleUnderstanding PyMongo - Comment Thanks for the tips!
Blog Post TitleMongoDB vs SQL Databases - Comment Great comparisons!


In [22]:
# If the number of comments per post grows large
# We can create another collection for comments
# {_id: string, user_name: string, text: string, timestamp: string, blog_id: string}

comments_collection = db.comments

comments_collection.delete_many({})

sample_posts = [
    {
        "title": "How to Use MongoDB",
        "content": "Using MongoDB effectively.",
        "author": "John",
        "tags": [retrieved_tags['MongoDB'], retrieved_tags['Beginner']]
    },
    {
        "title": "Understanding PyMongo",
        "content": "A guide to using PyMongo with MongoDB.",
        "author": "John",
        "tags": [retrieved_tags['PyMongo'], retrieved_tags['Programming']]
    },
    {
        "title": "Database Optimization",
        "content": "Techniques for optimizing database performance.",
        "author": "Alice",
        "tags": [retrieved_tags['Database'], retrieved_tags['Optimisation']]
    },
    {
        "title": "Schema Design in MongoDB",
        "content": "How to model your data in MongoDB.",
        "author": "Bob",
        "tags": [retrieved_tags['Schema'], retrieved_tags['MongoDB']]
    },
    {
        "title": "MongoDB vs SQL Databases",
        "content": "Comparing NoSQL and SQL databases.",
        "author": "Alice",
        "tags": [retrieved_tags['NoSQL'], retrieved_tags['SQL']]
    },
]

posts.delete_many({})

posts.insert_many(sample_posts)

retrieved_posts = {post['title']: post['_id'] for post in posts.find()}

sample_comments = [
    {
        "post_id": retrieved_posts["How to Use MongoDB"],
        "user_name": "Bob",
        "text": "Great post!",
        "timestamp": "2024-09-12T10:00"
    },
    {
        "post_id": retrieved_posts["Understanding PyMongo"],
        "user_name": "Alice",
        "text": "Very informative.",
        "timestamp": "2024-09-13T14:30"
    },
    {
        "post_id": retrieved_posts["Understanding PyMongo"],
        "user_name": "Bob",
        "text": "Thanks for the tips!",
        "timestamp": "2024-09-13T15:00"
    },
    {
        "post_id": retrieved_posts["Schema Design in MongoDB"],
        "user_name": "Alice",
        "text": "This is helpful!",
        "timestamp": "2024-09-14T09:45"
    },
    {
        "post_id": retrieved_posts["MongoDB vs SQL Databases"],
        "user_name": "Bob",
        "text": "Great comparisons!",
        "timestamp": "2024-09-15T08:20"
    },
    {
        "post_id": retrieved_posts["MongoDB vs SQL Databases"],
        "user_name": "John",
        "text": "Loved this article!",
        "timestamp": "2024-09-15T10:15"
    },
]

comments_collection.insert_many(sample_comments)

InsertManyResult([ObjectId('6910e4f24108134fdc255384'), ObjectId('6910e4f24108134fdc255385'), ObjectId('6910e4f24108134fdc255386'), ObjectId('6910e4f24108134fdc255387'), ObjectId('6910e4f24108134fdc255388'), ObjectId('6910e4f24108134fdc255389')], acknowledged=True)